Project guide: https://www.dataquest.io/m/216/guided-project%3A-creating-relations-in-sqlite

Solution by DataQuest: https://github.com/dataquestio/solutions/blob/master/Mission216Solutions.ipynb

<br /><br />

This uses a copy of `nominations.db`, a database produced in the project [PreparingDataForSQLite](../project1/PreparingDataForSQLite.ipynb).

`nominations.db` is already in the directory. A copy will be made and named as `nominations2.db`.

This project modifies the database. Therefore, a copy is used to allow running the codes more than once.

# 1. Introduction to the data

The only table in `nominations.db` is `nominations`. Its schema is shown below.

In [1]:
import sqlite3
from shutil import copy

# make a copy of "nominations.db"
copy("nominations.db", "nominations2.db")

# use the copy from this point on
db = "nominations2.db"

def runSqlite(query, db=db, inputList=None, foreignKey=False, returnOutput=False):
    """
    Connect to "nominations.db",
    execute SQLite query,
    print output row by row,
    and close connection.
    
    If inputList is provided,
    its elements will be iteratively fed into query.
    
    If foreignKey is True,
    foreign key contrains will be turned on.
    
    If returnOutput is True,
    output of cursor.fetchall() will be returned.
    """
    
    # connect to database
    conn = sqlite3.connect(db)
    
    # create cursor
    cursor = conn.cursor()
    
    # turn on foreign key constraint if required
    if foreignKey:
        cursor.execute("PRAGMA foreign_keys = ON")
    
    # execute query
    if inputList is not None:
        try:
            cursor.executemany(query, inputList)
        except:
            conn.close()
            raise
    else:
        try:
            cursor.execute(query)
        except:
            conn.close()
            raise
    
    output = cursor.fetchall()

    # return output if required
    if returnOutput:

        # close connection
        conn.close()
        
        return output
    
    # for PRAGMA or SELECT queries: print each row of output
    query_lo = query.lower()
    if query_lo.startswith(("pragma", "select")):
        for row in output:
            print(row)
    # for INSERT or UPDATE queries: save changes
    elif query_lo.startswith(("insert", "update")):
        conn.commit()

    # close connection
    conn.close()

# show schema of "nominations" table
query = "PRAGMA table_info(nominations)"
runSqlite(query)

(0, 'Year', 'INTEGER', 0, None, 0)
(1, 'Category', 'TEXT', 0, None, 0)
(2, 'Nominee', 'TEXT', 0, None, 0)
(3, 'Won', 'INTEGER', 0, None, 0)
(4, 'Movie', 'TEXT', 0, None, 0)
(5, 'Character', 'TEXT', 0, None, 0)


Following are first 10 rows of `nominations` table.

In [2]:
# 2. show top 10 rows
query = "SELECT * FROM nominations LIMIT 10"
runSqlite(query)

(2010, 'Actor -- Leading Role', 'Javier Bardem', 0, 'Biutiful', 'Uxbal')
(2010, 'Actor -- Leading Role', 'Jeff Bridges', 0, 'True Grit', 'Rooster Cogburn')
(2010, 'Actor -- Leading Role', 'Jesse Eisenberg', 0, 'The Social Network', 'Mark Zuckerberg')
(2010, 'Actor -- Leading Role', 'Colin Firth', 1, "The King's Speech", 'King George VI')
(2010, 'Actor -- Leading Role', 'James Franco', 0, '127 Hours', 'Aron Ralston')
(2010, 'Actor -- Supporting Role', 'Christian Bale', 1, 'The Fighter', 'Dicky Eklund')
(2010, 'Actor -- Supporting Role', 'John Hawkes', 0, "Winter's Bone", 'Teardrop')
(2010, 'Actor -- Supporting Role', 'Jeremy Renner', 0, 'The Town', 'James Coughlin')
(2010, 'Actor -- Supporting Role', 'Mark Ruffalo', 0, 'The Kids Are All Right', 'Paul')
(2010, 'Actor -- Supporting Role', 'Geoffrey Rush', 0, "The King's Speech", 'Lionel Logue')


# 2. Demonstrate one-to-many relationship between tables

`nominations` will be the "many" and `ceremonies` one. The latter will be created now.

# 2.1. Create `ceremonies` table.

`ceremonies` table will be added to `nominations.db`. It will include three columns.

* `id` - Unique identifier for each row
* `Year` - Year of ceremony
* `Host` - Host of ceremony

Data for this table is from https://en.wikipedia.org/wiki/List_of_Academy_Awards_ceremonies#Ceremonies. There were two hosts in 2010 ceremony, but only one will be kept for simplicity.

Now, `ceremonies` table will be created and its schema displayed.

In [3]:
# years and hosts
years_hosts = [(2010, "Steve Martin"),
               (2009, "Hugh Jackman"),
               (2008, "Jon Stewart"),
               (2007, "Ellen DeGeneres"),
               (2006, "Jon Stewart"),
               (2005, "Chris Rock"),
               (2004, "Billy Crystal"),
               (2003, "Steve Martin"),
               (2002, "Whoopi Goldberg"),
               (2001, "Steve Martin")
              ]

# remove "ceremonies" table (if existing from previous run)
query = "DROP TABLE IF EXISTS ceremonies"
runSqlite(query)

# create "ceremonies" table
query = \
"CREATE TABLE ceremonies(\
    id INTEGER PRIMARY KEY, \
    Year INTEGER, \
    Host TEXT)"

runSqlite(query)

# display schema of "ceremonies" table
query = "PRAGMA table_info(ceremonies)"
runSqlite(query)

(0, 'id', 'INTEGER', 0, None, 1)
(1, 'Year', 'INTEGER', 0, None, 0)
(2, 'Host', 'TEXT', 0, None, 0)


The `ceremonies` table will be filled and its first 10 rows displayed below.

In [4]:
# fill "ceremonies" table
query = "INSERT INTO ceremonies (Year, Host) VALUES(?, ?)"
runSqlite(query, inputList=years_hosts)

query = "SELECT * FROM ceremonies LIMIT 10"
runSqlite(query)

(1, 2010, 'Steve Martin')
(2, 2009, 'Hugh Jackman')
(3, 2008, 'Jon Stewart')
(4, 2007, 'Ellen DeGeneres')
(5, 2006, 'Jon Stewart')
(6, 2005, 'Chris Rock')
(7, 2004, 'Billy Crystal')
(8, 2003, 'Steve Martin')
(9, 2002, 'Whoopi Goldberg')
(10, 2001, 'Steve Martin')


## 2.2. Setting up one-to-many

Next thing to do is to replace `Year` column in `nominations` table with a new column `ceremony_id`. The `ceremony_id` will be used as a foreign key linked to `id` of `ceremonies` table.

As SQLite does not allow deleting columns, the following will be done instead.

* Create a new table `nominations_two` which will be a copy of `nominations` with `ceremony_id` column added.
* Remove `nominations` table.
* Rename `nominations_two` table as `nominations`.

So, `nominations_two` is created below and its schema displayed.

<a name="one_to_many"></a>

In [5]:
# create "nominations_two" table
query = \
"CREATE TABLE IF NOT EXISTS nominations_two(\
    id INTEGER PRIMARY KEY, \
    category TEXT, \
    nominee TEXT, \
    movie TEXT, \
    character TEXT, \
    won INTEGER, \
    ceremony_id INTEGER, \
        FOREIGN KEY (ceremony_id) \
            REFERENCES ceremonies(id))"

runSqlite(query)

# show schema of "nominations_two"
query = "PRAGMA table_info(nominations_two)"
runSqlite(query)

(0, 'id', 'INTEGER', 0, None, 1)
(1, 'category', 'TEXT', 0, None, 0)
(2, 'nominee', 'TEXT', 0, None, 0)
(3, 'movie', 'TEXT', 0, None, 0)
(4, 'character', 'TEXT', 0, None, 0)
(5, 'won', 'INTEGER', 0, None, 0)
(6, 'ceremony_id', 'INTEGER', 0, None, 0)


Next, `nominations_two` gets filled and its first 10 rows gets displayed.

In [6]:
# get all columns of "nominations" table and 
# "id" column of "ceremonies" table
query = \
"SELECT nominations.Category, \
        nominations.Nominee, \
        nominations.Movie, \
        nominations.Character, \
        nominations.Won, \
        ceremonies.id \
FROM nominations \
INNER JOIN ceremonies \
    ON nominations.Year == ceremonies.Year"

joined_nominations = runSqlite(query, returnOutput=True)

# insert the output into "nominations_two" table
query = \
"INSERT INTO nominations_two \
    (category, nominee, movie, character, won, ceremony_id) \
    VALUES(?, ?, ?, ?, ?, ?)"

runSqlite(query, inputList=joined_nominations, foreignKey=True)

# display first 10 rows.
query = "SELECT * FROM nominations_two LIMIT 10"
runSqlite(query)

(1, 'Actor -- Leading Role', 'Javier Bardem', 'Biutiful', 'Uxbal', 0, 1)
(2, 'Actor -- Leading Role', 'Jeff Bridges', 'True Grit', 'Rooster Cogburn', 0, 1)
(3, 'Actor -- Leading Role', 'Jesse Eisenberg', 'The Social Network', 'Mark Zuckerberg', 0, 1)
(4, 'Actor -- Leading Role', 'Colin Firth', "The King's Speech", 'King George VI', 1, 1)
(5, 'Actor -- Leading Role', 'James Franco', '127 Hours', 'Aron Ralston', 0, 1)
(6, 'Actor -- Supporting Role', 'Christian Bale', 'The Fighter', 'Dicky Eklund', 1, 1)
(7, 'Actor -- Supporting Role', 'John Hawkes', "Winter's Bone", 'Teardrop', 0, 1)
(8, 'Actor -- Supporting Role', 'Jeremy Renner', 'The Town', 'James Coughlin', 0, 1)
(9, 'Actor -- Supporting Role', 'Mark Ruffalo', 'The Kids Are All Right', 'Paul', 0, 1)
(10, 'Actor -- Supporting Role', 'Geoffrey Rush', "The King's Speech", 'Lionel Logue', 0, 1)


The old `nominations` will be removed. Then, `nominations_two` will be renamed as `nominations`. Finally, I will verify the outcome by displaying details of all tables in the database.

In [7]:
# remove "nominations" table
query = "DROP TABLE nominations"
runSqlite(query)

# rename "nominations_two" table as "nominations"
query = \
"ALTER TABLE nominations_two \
RENAME TO nominations"

runSqlite(query)

# verify the renaming - list of table details
query = "SELECT * FROM sqlite_master WHERE type='table'"
runSqlite(query)

('table', 'ceremonies', 'ceremonies', 7, 'CREATE TABLE ceremonies(    id INTEGER PRIMARY KEY,     Year INTEGER,     Host TEXT)')
('table', 'nominations', 'nominations', 8, 'CREATE TABLE "nominations"(    id INTEGER PRIMARY KEY,     category TEXT,     nominee TEXT,     movie TEXT,     character TEXT,     won INTEGER,     ceremony_id INTEGER,         FOREIGN KEY (ceremony_id)             REFERENCES ceremonies(id))')


# 3. Demonstrate many-to-many relationship between tables

I will create two tables first, named as `movies` and `actors`. Each will contain `movie` and `nominee` columns, respectively, from the `nominations` table. Then, I will create a `join table` to link the two in many-to-many relationship.

First, `movies` table will be created and filled. Its first 10 rows will be displayed.

In [8]:
# create "movies" table
query = \
"CREATE TABLE IF NOT EXISTS movies (\
    id INTEGER PRIMARY KEY, \
    movie TEXT)"

runSqlite(query)

# fill table with "movie" column of "nominations" table
query = \
"INSERT INTO movies (movie) \
    SELECT DISTINCT movie FROM nominations"

runSqlite(query)

# display first 10 rows of "movies" table
query = "SELECT * FROM movies LIMIT 10"
runSqlite(query)

(1, 'Biutiful')
(2, 'True Grit')
(3, 'The Social Network')
(4, "The King's Speech")
(5, '127 Hours')
(6, 'The Fighter')
(7, "Winter's Bone")
(8, 'The Town')
(9, 'The Kids Are All Right')
(10, 'Rabbit Hole')


The same will be done for the `actors` table.

In [9]:
# create "actors" table
query = \
"CREATE TABLE IF NOT EXISTS actors (\
    id INTEGER PRIMARY KEY, \
    actor TEXT)"

runSqlite(query)

# fill table with "movie" column of "nominations" table
query = \
"INSERT INTO actors (actor) \
    SELECT DISTINCT nominee FROM nominations"

runSqlite(query)

# display first 10 rows of "movies" table
query = "SELECT * FROM actors LIMIT 10"
runSqlite(query)

(1, 'Javier Bardem')
(2, 'Jeff Bridges')
(3, 'Jesse Eisenberg')
(4, 'Colin Firth')
(5, 'James Franco')
(6, 'Christian Bale')
(7, 'John Hawkes')
(8, 'Jeremy Renner')
(9, 'Mark Ruffalo')
(10, 'Geoffrey Rush')


Finally, the `join table` will be created and filled. Then, its first 10 rows will be displayed.

<a name="join_table"></a>

In [10]:
# 1. create join table "movies_actors"
query = \
"CREATE TABLE IF NOT EXISTS movies_actors (\
    id INTEGER PRIMARY KEY, \
    movie_id INTEGER REFERENCES movies(id), \
    actor_id INTEGER REFERENCES actors(id))"

runSqlite(query)

# 2. fill join table

# 2.1. get movies and nominees from "nominations" table
query = \
"SELECT Movie, Nominee \
FROM nominations"

movies_nominees = runSqlite(query, returnOutput=True)

# 2.2. fill join table with the movies and nominees (i.e. actors)
query = \
"INSERT INTO movies_actors (movie_id, actor_id) \
    VALUES(\
        (SELECT id FROM movies WHERE movie == ?), \
        (SELECT id FROM actors WHERE actor == ?))"

runSqlite(query, inputList=movies_nominees, foreignKey=True)

# 3. display first 10 rows of join table
query = "SELECT * FROM movies_actors LIMIT 10"
runSqlite(query)

(1, 1, 1)
(2, 2, 2)
(3, 3, 3)
(4, 4, 4)
(5, 5, 5)
(6, 6, 6)
(7, 7, 7)
(8, 8, 8)
(9, 9, 9)
(10, 4, 10)


# 6. Next steps

Following questions are asked by DataQuest as further steps. My answers are provided for each question.

## What other datasets can we add to the database?

Demographics (e.g. age, gender) of nominees could be added in a separate table `demographics`. Then, a one-to-many relationship can be established (`demographics` as "one" and `nominations` as "many").

## Based on what you know, brainstorm how you would populate the join table and the linked tables using data from `nominations`.

These have been carried out above ([one-to-many](#one_to_many) and [many-to-many (join table)](#join_table)) - for many-to-many relationship, I referred to DataQuest's solution (go [here](https://github.com/dataquestio/solutions/blob/master/Mission216Solutions.ipynb) and see last section ("Populating a join table")).

For both cases, following were done.

* Relevant columns were extracted from `nominations`. The output was saved as a list of tuples. Let's refer to this as `output` for convenience.
* Then, an INSERT query was built with a place holder for per column, which would be iteratively filled with each row of `output`.
* Lastly, the query was executed with `sqlite3`'s [`executemany`](https://docs.python.org/3/library/sqlite3.html#sqlite3.Cursor.executemany) method.